# Strands Agent Graph로 멀티 에이전트 시스템 구축
멀티 에이전트 시스템은 여러 전문화된 AI 에이전트가 협력하여 복잡한 문제를 해결하도록 함께 작동합니다. 각 에이전트는 명시적인 통신 경로를 통해 연결된 특정 기능과 역할을 가지고 있습니다.

이 실습에서는 Strands Agent SDK를 사용하여 멀티 에이전트 시스템을 구축하는 방법을 배웁니다. 기본 개념에서 고급 구현까지 진행하면서 다양한 토폴로지와 실제 애플리케이션을 탐색합니다.

**학습 목표:**
이 노트북을 마치면 다음을 수행할 수 있습니다:
- 에이전트 그래프의 세 가지 핵심 구성 요소(노드, 엣지, 조건) 이해
- 특정 에이전트 간에 타겟 메시지 전송
- 멀티 에이전트 네트워크 모니터링 및 제어
- 실제 시나리오를 위한 전문화된 에이전트 시스템 설계

## 사전 요구사항

- Python 3.10+
- Amazon Bedrock에서 활성화된 Anthropic Claude 3.7이 있는 AWS 계정
- Amazon Bedrock을 사용할 수 있는 권한이 있는 IAM 역할
- AI 에이전트 및 프롬프트 엔지니어링에 대한 기본 이해

## 설정 및 설치

시작하기 전에 `strands-agents`와 `strands-agents-tools`에 필요한 패키지를 설치하겠습니다

In [1]:
!pip install -r requirements.txt

### 필요한 패키지 가져오기

다음으로 필요한 패키지를 가져올 수 있습니다

In [2]:
from strands import Agent

## 에이전트 그래프 구성 요소 이해
에이전트 그래프는 조정된 협업을 통해 복잡한 문제를 해결하도록 설계된 상호 연결된 AI 에이전트의 구조화된 네트워크입니다. 각 에이전트는 특정 기능을 가진 전문화된 노드를 나타내며, 에이전트 간의 연결은 명시적인 통신 경로를 정의합니다.

구축을 시작하기 전에 에이전트 그래프의 세 가지 주요 구성 요소를 이해해 봅시다:

### 1. 노드 (Agents)
각 노드는 다음을 가진 AI 에이전트를 나타냅니다:
- **Identity**: 그래프 내의 고유 식별자
- **Role**: 전문화된 기능 또는 목적
- **System Prompt**: 에이전트의 동작을 정의하는 지침
- **Tools**: 에이전트에서 사용할 수 있는 기능

### 2. 엣지 (Connections)
엣지는 다음을 가진 통신 경로를 정의합니다:
- **Direction**: 단방향 또는 양방향 정보 흐름
- **Condition**: 엣지를 통과해야 하는지 결정하는 선택적 함수
- **Dependencies**: 노드 간의 실행 순서 및 데이터 흐름 정의

### 3. GraphBuilder
GraphBuilder는 그래프 구축을 위한 간단한 인터페이스를 제공합니다:
- **add_node()**: 에이전트 또는 멀티 에이전트 시스템을 노드로 추가
- **add_edge()**: 노드 간 종속성 생성
- **set_entry_point()**: 실행을 위한 시작 노드 정의
- **build()**: Graph 인스턴스 검증 및 생성


### 기본 처리

두 개의 다른 에이전트가 처리한 하나의 작업의 간단한 예제부터 시작하겠습니다. 정의된 역할에 따라 출력이 달라집니다. 노드의 실행 순서를 살펴보고 Strands SDK를 사용하면 필요한 경우 단일 노드에서만 명시적으로 응답을 얻을 수 있다는 점도 확인하세요. 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/basic.png" width="55%" />
</div>

In [4]:
# 에이전트 그래프 기능을 가진 에이전트 초기화
from strands.multiagent import GraphBuilder

# 전문 에이전트 생성
coordinator = Agent(name="coordinator", system_prompt="당신은 전문가들을 조정하는 연구팀 리더입니다. 간단한 분석을 제공하세요. 추가 질문은 필요 없습니다.")
analyst = Agent(name="data_analyst", system_prompt="당신은 통계 분석을 전문으로 하는 데이터 분석가입니다. 간단한 분석을 제공하세요. 추가 질문은 필요 없습니다")
domain_expert = Agent(name="domain_expert", system_prompt="당신은 해당 분야에 대한 깊은 지식을 가진 전문가입니다. 간단한 분석을 제공하세요. 추가 질문은 필요 없습니다")

# 그래프 구축
builder = GraphBuilder()

# 노드 추가
builder.add_node(coordinator, "team_lead")
builder.add_node(analyst, "analyst")
builder.add_node(domain_expert, "expert")

# 에지(의존성) 추가
builder.add_edge("team_lead", "analyst")
builder.add_edge("team_lead", "expert")

# 진입점 설정 (선택사항 - 지정하지 않으면 자동 감지됨)
builder.set_entry_point("team_lead")

# 그래프 생성
graph = builder.build()

# 새로 생성된 그래프에서 작업 실행
result = graph("원격 근무가 직원 생산성에 미치는 영향 분석. 간략한 분석 제공, 후속 조치 불필요")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:==========================")
print("============================================================")

# 실행된 노드와 실행 순서 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭 가져오기
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드: {result.completed_nodes}")
print(f"실패한 노드: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")


# 특정 노드 결과 가져오기
print("\n")
print("=============전문가 노드 결과만:======================")
print("============================================================")
print(result.results["expert"].result)

Graph without execution limits may run indefinitely if cycles exist


# 원격 근무의 직원 생산성 영향 분석

## 🔍 핵심 발견사항

### 긍정적 영향
- **생산성 향상**: 67%의 기업이 원격 근무 도입 후 직원 생산성 증가 보고
- **집중력 개선**: 사무실 방해 요소 제거로 깊은 작업 시간 35% 증가
- **업무-생활 균형**: 통근 시간 절약으로 실질 근무 시간 효율성 향상

### 부정적 영향
- **협업 한계**: 즉석 소통 감소로 프로젝트 완료 시간 평균 15% 지연
- **고립감**: 직원 28%가 동기 부여 저하 및 소속감 약화 경험
- **홈오피스 환경**: 23%의 직원이 집중력 저하 요인(가족, 생활 소음) 보고

## 📊 종합 평가

**전체적으로 원격 근무는 생산성에 긍정적 영향을 미치나, 업무 유형과 개인 특성에 따라 편차 존재**

- **개인 작업 중심**: 생산성 증가 효과 뚜렷
- **팀 협업 중심**: 생산성 저하 위험 상존
- **하이브리드 모델**: 양쪽 장점을 활용한 최적 대안으로 평가

## 💡 권고사항
1. 업무 특성별 맞춤형 원격 근무 정책 수립
2. 디지털 협업 도구 고도화
3. 정기적 성과 모니터링 체계 구축# 원격 근무 #생산성 영향 통계 분석

## 📈 원격 근무의 생산성 영향: 핵심 통계 지표

### 생 전문가 분석

## 📈산성 변화 수치
- 데이터 기반 핵심 통찰

** **긍정적 영향**: 67% 기업이 생산성 증가 보고
- **집전체 생산성 지표**: 원중력 향상**: 깊은 작업 시간 35% 증가
- **부정적 영향**: 프격 근무는 **순증 효과**를 보이며, 특히 지로젝트 완료 시간 15% 지연

### 직원 경식 기반 업무에서 13-험 분포
- **동기 부여 저하**: 28% 직원
- **집50% 생산성 향상이 측중력 저하**: 23% 직원 (정됨

### 🎯 생산성 향홈오피스 환경 요인)
- **만족도**:상 메커니즘
- ** 약 72% 직원이 전반적 긍정 평딥워크 시간 증대가

## 🎯 **: 방해 요소 차단으로 고통계적 결론품질 작업 시간 확보

### 병렬 처리

이제 문제의 두 가지 다른 측면을 살펴보는 2개의 에이전트가 요청을 처리하고 제공된 입력을 기반으로 요약 및 위험 계산을 담당하는 최종 에이전트에 입력하는 토폴로지를 만들어 보겠습니다
<div style="text-align:left">
    <img src="images/parallel.png" width="55%" />
</div>

In [5]:
# agent_graph 기능을 가진 에이전트 초기화
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# 특수화된 에이전트 생성

financial_advisor = Agent(name="financial_advisor", system_prompt="당신은 비용-편익 분석, 예산 영향, ROI 계산에 중점을 둔 재무 고문입니다. 다른 전문가들과 협력하여 포괄적인 재무 관점을 구축하십시오.")
technical_architect = Agent(name="technical_architect", system_prompt="당신은 타당성, 구현 과제, 기술적 위험을 평가하는 기술 아키텍트입니다. 다른 전문가들과 협력하여 기술적 실현 가능성을 확보하십시오.")
market_researcher = Agent(name="market_researcher", system_prompt="당신은 시장 상황, 사용자 요구, 경쟁 환경을 분석하는 시장 조사원입니다. 다른 전문가들과 협력하여 시장 기회를 검증하십시오.")
risk_analyst = Agent(name="risk_analyst", system_prompt="당신은 잠재적 위험, 완화 전략 및 규정 준수 문제를 식별하는 리스크 애널리스트입니다. 포괄적인 위험 평가를 보장하기 위해 다른 전문가들과 협력하십시오.")


# 그래프 생성
builder = GraphBuilder()

# 노드 추가
builder.add_node(financial_advisor, "finance_expert")
builder.add_node(technical_architect, "tech_expert")
builder.add_node(market_researcher, "market_expert")
builder.add_node(risk_analyst, "risk_analyst")

# 에지(의존성) 추가
builder.add_edge("finance_expert", "tech_expert")
builder.add_edge("finance_expert", "market_expert")
builder.add_edge("tech_expert", "risk_analyst")
builder.add_edge("market_expert", "risk_analyst")


# 진입점 설정 (선택 사항 - 지정하지 않으면 자동 감지됨)
builder.set_entry_point("finance_expert")

# 그래프 생성
graph = builder.build()

print("============================================================")
print("============================================================")

# 새로 생성된 그래프에서 작업 실행
result = graph("우리 회사는 새로운 AI 기반 고객 서비스 플랫폼 출시를 고려 중입니다. 초기 투자액은 200만 달러이며, 3년 예상 ROI는 150%입니다. 재무적 평가를 알려주세요.")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:==========================")
print("============================================================")

# 실행된 노드와 실행 순서 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭 가져오기
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드: {result.completed_nodes}")
print(f"실패한 노드: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")


# 특정 노드 결과 가져오기

print("금융 전문가:")
print("============================================================")
print("============================================================")
print(result.results["finance_expert"].result)
print("\n")

print("기술 전문가:")
print("============================================================")
print("============================================================")
print(result.results["tech_expert"].result)
print("\n")

print("시장 조사 전문가:")
print("============================================================")
print("============================================================")
print(result.results["market_expert"].result)
print("\n")

Graph without execution limits may run indefinitely if cycles exist


# AI 기반 고객 서비스 플랫폼 재무 분석

## 📊 **핵심 재무 지표 분석**

### **ROI 및 수익성 평가**
- **총 투자 수익**: $3,000,000 (150% ROI 기준)
- **순 수익**: $1,000,000
- **연간 평균 ROI**: 50%
- **투자 회수 기간**: 약 2년

## 💰 **비용-편익 분석**

### **예상 비용 구조**
```
초기 투자: $2,000,000
├── 시스템 구축: $1,200,000 (60%)
├── 라이선스/소프트웨어: $400,000 (20%)
├── 교육/구현: $300,000 (15%)
└── 예비비: $100,000 (5%)

연간 운영비: 약 $200,000-300,000
```

### **예상 편익**
- **인건비 절감**: 연간 $400,000-600,000
- **처리 속도 향상**: 고객 만족도 증가로 인한 매출 증가
- **24/7 서비스**: 기회비용 감소

## 📈 **예산 영향 분석**

### **현금흐름 예측** (3개년)
| 연도 | 투자비용 | 운영비용 | 예상수익 | 순현금흐름 | 누적현금흐름 |
|------|----------|----------|----------|------------|--------------|
| 1년차 | $2,000,000 | $250,000 | $800,000 | -$1,450,000 | -$1,450,000 |
| 2년차 | $0 | $275,000 | $1,200,000 | $925,000 | -$525,000 |
| 3년차 | $0 | $300,000 | $1,500,000 | $1,200,000 | $675,000 |

## ⚠️ **리스크 요소 및 민감도 분석**

### **주요 재무 리스크**
1. **기술 변화 속도**: AI 기술 급속 발전으로 인한 조기 노후화
2. **구현 지연**: 예산 초과 및 ROI 지연 가능성
3. **고객 적응**: 도입 초기 고객 저항으로 인한 수익 지연

### **시나리오

### 조건부 분기

요청을 분류하고 코드에서 정의한 조건에 따라 요청을 기술 또는 비즈니스 에이전트로 라우팅하는 에이전트 그래프를 만들어 보겠습니다.

두 가지 다른 프롬프트를 기반으로 한 이 그래프의 노드 실행 순서와 실행된 노드 수의 차이를 자세히 살펴보세요.

<div style="text-align:left">
    <img src="images/conditional.png" width="55%" />
</div>

In [7]:
# 에이전트 그래프 기능을 가진 에이전트 초기화
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# 특수화된 에이전트 생성

classifier = Agent(name="분류기", system_prompt="보고서 요청을 기술적 또는 비즈니스적 분류로만 반환하는 분류 담당 에이전트입니다.")
technical_report = Agent(name="technical_expert", system_prompt="당신은 기술적 관점에서 간략한 요약 제공에 집중하는 기술 전문가입니다.")
business_report = Agent(name="business_expert", system_prompt="당신은 비즈니스 관점에서 간략한 요약 제공에 집중하는 비즈니스 전문가입니다.")

# 그래프 구축
builder = GraphBuilder()

# 노드 추가
builder.add_node(classifier, "classifier")
builder.add_node(technical_report, "technical_report")
builder.add_node(business_report, "business_report")

def is_technical(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "technical" in result_text.lower()

def is_business(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "business" in result_text.lower()

# 에지 추가 (의존성)
builder.add_edge("classifier", "technical_report", condition=is_technical)
builder.add_edge("classifier", "business_report", condition=is_business)

# 엔트리 포인트 설정 (선택 사항 - 지정하지 않으면 자동 감지됨)
builder.set_entry_point("classifier")

# 그래프 생성
graph = builder.build()

print("============================================================")
print("============================================================")

# 새로 생성된 그래프에서 작업 실행
result = graph("재택근무의 기술적 측면에 대한 보고서를 제공하고, 고려사항과 주요 위험 요소를 개요로 제시하라")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:==========================")
print("============================================================")

# 실행된 노드와 순서 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭 가져오기
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드: {result.completed_nodes}")
print(f"실패한 노드: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")

# 특정 노드에서 결과 가져오기

print("분류기:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

# 새로 생성된 그래프에서 작업 실행
result = graph("재택근무의 비즈니스 영향에 대한 보고서를 제공하고, 고려 사항 및 주요 위험 요소를 개요로 제시하라")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:==========================")
print("============================================================")

# 실행된 노드와 실행 순서 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭스 확인
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드: {result.completed_nodes}")
print(f"실패한 노드: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")

# 특정 노드 결과 가져오기

print("분류기:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

Graph without execution limits may run indefinitely if cycles exist


**분류: 기술적**

이 요청은 재택근무의 기술적 인프라, 보안, 도구 및 시스템에 관한 기술적 분석을 요구하고 있습니다.

**재택근무 기술적 측면 보고서**

## 1. 핵심 기술 인프라

### 네트워크 연결성
- **광대역 인터넷**: 최소 25Mbps 다운로드/3Mbps 업로드 권장
- **백업 연결**: 모바일 핫스팟 또는 이중화 ISP
- **VPN 솔루션**: 기업 네트워크 안전 접근

### 하드웨어 요구사항
- **컴퓨팅 장비**: 노트북/데스크톱 (최소 8GB RAM, SSD)
- **주변기기**: 웹캠, 마이크, 듀얼 모니터
- **보안 장비**: 하드웨어 토큰, 생체인식 장치

## 2. 소프트웨어 생태계

### 협업 도구
- **화상회의**: Zoom, Teams, Google Meet
- **프로젝트 관리**: Asana, Jira, Trello
- **파일 공유**: OneDrive, Google Drive, Dropbox

### 보안 솔루션
- **엔드포인트 보호**: EDR/XDR 솔루션
- **ID 관리**: SSO, MFA 구현
- **데이터 보호**: 암호화, DLP 정책

## 3. 주요 기술적 고려사항

### 성능 최적화
- 대역폭 관리 및 QoS 설정
- 클라우드 서비스 지연시간 최소화
- 캐싱 및 CDN 활용

### 확장성
- 사용자 증가에 따른 인프라 스케일링
- 라이선스 관리 자동화
- 클라우드 기반 탄력적 리소스

## 4. 핵심 위험 요소

### 보안 위험 (높음)
- **데이터 유출**: 비보안 네트워크 사용
- **멀웨어 감염**: 개인 장비의 보안 취약점
- **피싱 공격**: 원격 환경에서의 사회공학적 공격

### 기술적 위험 (중간)
- **연결 불안정**: 인터넷 서비스 중단
- **하드웨어 장애**: 개인 장비 고장 시 업무 중단
- **소프트웨어 호환성**: 버전 불일치 및 업데이트 문제

### 운영 위험 (중간)
- **기술 지원**: 원격 IT 지원의 제약
- **백업/복구**

## 주요 요점 및 모범 사례


### 모범 사례:

**비순환성을 위한 설계:** 그래프에 순환이 없는지 확인</p>
**의미 있는 노드 ID 사용:** 노드에 대한 설명적인 이름 선택</p>
**그래프 구조 검증:** 빌더가 순환을 확인하고 진입점을 검증</p>
**노드 실패 처리:** 한 노드의 실패가 전체 워크플로에 미치는 영향 고려</p>
**조건부 엣지 사용:** 중간 결과를 기반으로 한 동적 워크플로를 위해</p>
**병렬성 고려:** 독립적인 분기는 동시에 실행 가능</p>
**멀티 에이전트 패턴 중첩:** 복잡한 워크플로를 위해 그래프 내에서 Swarms 사용</p>
**멀티모달 입력 활용:** 이미지를 포함한 풍부한 입력을 위해 ContentBlocks 사용</p>

## 결론

이제 Strands Agent Graph로 멀티 에이전트 시스템을 구축하는 기본 사항을 마스터했습니다! 복잡한 문제를 해결하기 위해 협력하는 전문화된 AI 에이전트의 정교한 네트워크를 만들 수 있습니다.

성공적인 멀티 에이전트 시스템의 핵심은 다음과 같습니다:
- 사용 사례에 맞는 토폴로지 매칭
- 명확한 에이전트 역할 및 책임 정의
- 적절한 통신 패턴 구축
- 리소스 및 정리를 효과적으로 관리

여기에서부터 연구, 콘텐츠 생성, 의사 결정, 고객 서비스 등의 실제 애플리케이션을 위해 점점 더 정교한 시스템을 구축할 수 있습니다.